In [1]:
import f3dasm
import numpy as np
import time
import pathos

2022-10-03 15:52:47.498514: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
dim = 5
iterations = 50
realizations = 3
bounds = np.tile([-1.0,1.0], (dim, 1))
#hyperparameters={'learning_rate': 2e-4}
hyperparameters={'learning_rate': 1e-2}

design = f3dasm.make_nd_continuous_design(bounds=bounds, dimensionality=dim)

function = f3dasm.functions.Levy(dimensionality=dim, scale_bounds=bounds)

data = f3dasm.Data(design=design)
optimizer = f3dasm.optimization.RMSprop(data=data, hyperparameters=hyperparameters)
sampler = f3dasm.sampling.LatinHypercubeSampling(design=design)


# all_data = f3dasm.run_multiple_realizations(optimizer=optimizer, 
# function=function,
# sampler=sampler,
# iterations=iterations,
# realizations=realizations
# )



In [3]:
optimizer.parameter

RMSprop_Parameters(population=1, force_bounds=True, learning_rate=0.01, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False)

In [4]:
args = {
'optimizer': optimizer, 
'function': function,
'sampler': sampler,
'iterations': iterations,
'seed': 1


}

#res = f3dasm.run_optimization(**args)

In [5]:
# function.plot_data(data=res,domain=bounds)

In [6]:
# args = {
# 'realizations': realizations,
# 'optimizer': optimizer, 
# 'function': function,
# 'sampler': sampler,
# 'iterations': iterations,
# 'parallelization': True


# }
# f3dasm.run_multiple_realizations(**args)

In [7]:
# """Run multiple realizations of the same algorithm on a benchmark function"""
# start_t = time.perf_counter()

# seed = np.random.randint(low=0, high=1e5)
# all_data = []

# for _ in range(realizations):
#     data = f3dasm.run_optimization(
#         optimizer=optimizer, function=function, sampler=sampler, iterations=iterations, seed=seed
#     )
#     all_data.append(data)

#     # Increase seed
#     seed += 1

# end_t = time.perf_counter()

# total_duration = end_t - start_t
# print(f"Optimization took {total_duration:.2f}s total")

In [8]:
from pathos.helpers import mp

In [9]:
"""Run multiple realizations of the same algorithm on a benchmark function"""
start_t = time.perf_counter()

args = [(optimizer, function,  sampler, iterations, np.random.randint(low=0, high=1e5)) for _ in range(realizations)]



with mp.Pool() as pool:
    results = pool.starmap(f3dasm.run_optimization, args)
    
end_t = time.perf_counter()

total_duration = end_t - start_t
print(f"Optimization took {total_duration:.2f}s total")

2022-10-03 15:52:50.029249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 15:52:50.029249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 15:52:50.033016: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations: 

Optimization took 1.13s total


In [10]:
results

[Data(design=DesignSpace(input_space=[ContinuousParameter(name='x0', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x1', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x2', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x3', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x4', lower_bound=-1.0, upper_bound=1.0, type='float')], output_space=[ContinuousParameter(name='y', lower_bound=-inf, upper_bound=inf, type='float')]), data=       input                                              output
           x0        x1        x2        x3        x4           y
 0  -0.785222  0.630130 -0.880795  0.482985  0.375636   77.395080
 1   0.279721  0.846332  0.601446  0.769809  0.670736  223.667528
 2   0.248099  0.877955  0.633069  0.738187  0.702359  166.207174
 3   0.216878  0.900263  0.660930  0.717253  0.697195  119.904358
 4   0.190026  0.916211  0.684178  0.703525  0.697016   86.7

In [11]:
"""Run multiple realizations of the same algorithm on a benchmark function"""
start_t = time.perf_counter()

args = [(optimizer, function,  sampler, iterations, np.random.randint(low=0, high=1e5)) for _ in range(realizations)]


with Pool() as pool:
    results = pool.starmap_async(f3dasm.run_optimization, args)
    
end_t = time.perf_counter()

total_duration = end_t - start_t
print(f"Optimization took {total_duration:.2f}s total")

NameError: name 'Pool' is not defined

In [ ]:
results

In [ ]:
func.dfdx(np.array([0.5, 0.8]))

In [ ]:
if dim == 2:
    fig, ax = func.plot_data(all_data[0], px=100, domain=bounds)
    ax.scatter(func.get_global_minimum(dim)[0][0],func.get_global_minimum(dim)[0][1], s=40, c='r')
    #ax.scatter(-3.19468541,0.89682456, s=40, c='g')
    func.plot(px=100, domain=bounds)

In [ ]:
y = np.array([[0.2,0.1],[0.3,0.4]])
y + np.abs(func.get_global_minimum(2)[1])

In [ ]:
all_data[0].data['input']

In [ ]:
func.scale_bounds, func.input_domain

In [ ]:
import pandas as pd
mean_y = pd.concat([d.get_output_data() for d in all_data], axis=1).mean(axis=1)
std_y = pd.concat([d.get_output_data() for d in all_data], axis=1).std(axis=1)
mean_y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.figure(), plt.axes()
ax.plot(mean_y)
ax.fill_between(np.arange(len(mean_y)), mean_y-1.96*std_y, mean_y+1.96*std_y, color='b', alpha=.1)
#ax.set_yscale('log')
fig.show()

In [ ]:
func.get_global_minimum(2)

In [ ]:
func.descale_input(np.array([0.0,0.0]))
x = np.array([[0.0,0.0]])

In [ ]:
func.input_domain

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0]) * x + func.input_domain[:, 0]

In [ ]:
(x - func.input_domain[:, 0]) / (func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
x

In [ ]:
func.input_domain[:, 0]

In [ ]:
x - func.input_domain[:, 0]

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
o = np.array([[0.9,0.3]])
o

In [ ]:
func.scale_input(func.descale_input(o))

In [ ]:
func.descale_input(func.scale_input(o))
o

In [ ]:
scale_bounds = [-3., 1.]

In [ ]:
o = np.random.uniform(low=scale_bounds[0], high=scale_bounds[1], size=(1, func.dimensionality))
x = o
x

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
(x - func.scale_bounds[:, 0]) / (func.scale_bounds[:, 1] - func.scale_bounds[0])

In [ ]:
func.offset

In [ ]:
func.get_global_minimum(2)[0]

In [ ]:
(func.scale_bounds[:, 1] - func.scale_bounds[0])